In [ ]:
import os
# os.environ['YOLO_VERBOSE'] = 'false'

In [ ]:
%pip install loguru==0.7.3 python-dotenv==1.0.1 PyYAML==6.0.2 torch==2.5.1 tqdm==4.67.1 typer==0.15.1 matplotlib==3.10.0 pyarrow==18.1.0 setuptools==75.1.0 protobuf==4.25.3 ultralytics==8.3.94 ray==2.43.0 albumentations==2.0.5 pandas

In [ ]:
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO, settings
import gc
import json
import locale
import os
import pandas as pd
import sys
import torch
import wandb
import yaml
from ultralytics.data.dataset import YOLODataset
from ultralytics.models.yolo.detect import DetectionTrainer, DetectionValidator
from ultralytics.utils import colorstr, LOGGER
import math
import numpy as np

sys.dont_write_bytecode = True
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Config

config_data = """
wandb:
  project: "EyeInTheSky_merged"
  group: "train"
data: "VisDrone.yaml"
# k_samples: 3
iterations: 1
train:
  project: "EyeInTheSky"
  data: "VisDrone.yaml"
  pretrained: True
  patience: 5
  task: detect
  epochs: 500
  seed: 42
  plots: True
  exist_ok: False
  save: True
  save_period: 10
  val: True
  warmup_epochs: 10
  visualize: True
  show: True
  single_cls: False
  rect: False
  resume: False
  fraction: 1.0
  freeze: None
  cache: False
  verbose: False
  amp: True
val:
  project: "EyeInTheSky"
  half: True
  conf: 0.25
  iou: 0.6
  split: "test"
  rect: True
  plots: True
  visualize: True
"""

In [ ]:
# Get device

def get_device() -> str:
    try:
        return 0 if torch.cuda.is_available() else "cpu"
    except Exception as e:
        print(f"Error setting device: {e}")

In [ ]:
# Load config

# config = Config.load("../config/config.yaml")
config = yaml.safe_load(config_data)
config["train"].update({"device" : get_device()})

In [ ]:
# Get Wandb key

def get_wandb_key_colab() -> str:
    try:
        from google.colab import userdata # type: ignore

        if userdata.get("WANDB_API_KEY") is not None:
            return userdata.get("WANDB_API_KEY")
        else:
            raise ValueError("No WANDB key found")
    except:
        return None

def get_wandb_env(path: Path) -> str:
    try:
        from dotenv import dotenv_values # type: ignore

        """Get W&B API key from Colab userdata or environment variable"""

        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Could not find .env file at {path}")

        print(f"Loading secrets from {path}")

        secrets = dotenv_values(path)
        print(f"Found keys: {list(secrets.keys())}")

        if "WANDB_API_KEY" not in secrets:
            raise KeyError(f"WANDB_API_KEY not found in {path}. Available keys: {list(secrets.keys())}")

        return secrets['WANDB_API_KEY']
    except:
        return None

def get_wandb_key(path: Path = "../.env") -> str:
    return get_wandb_key_colab() if get_wandb_key_colab() is not None else get_wandb_env(path)

In [ ]:
# Dataset, Trainer, Validator

class VisDroneDataset(YOLODataset):
    """
    Custom dataset for VisDrone that merges pedestrian (0) and people (1) classes.
    Handles class remapping at the earliest possible stage.
    """
    
    # Define the merged names as a class attribute to be accessible from the trainer
    merged_names = {
        0: 'persona',
        1: 'bicicletta',
        2: 'auto',
        3: 'furgone',
        4: 'camion',
        5: 'triciclo',
        6: 'triciclo-tendato',
        7: 'autobus',
        8: 'motociclo'
    }
    
    def __init__(self, *args, **kwargs):
        # Initialize parent class with modified kwargs
        super().__init__(*args, **kwargs)
        
        # Log class mapping
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Using merged classes: {self.merged_names}")
    
    def get_labels(self):
        """
        Load and process labels with class remapping.
        """
        # Get labels from parent method
        labels = super().get_labels()
        
        # Process statistics
        people_count = 0
        shifted_count = 0
        
        # Process labels to merge classes
        for i in range(len(labels)):
            cls = labels[i]['cls']
            
            if len(cls) > 0:
                # Count 'people' instances
                people_mask = cls == 1
                people_count += np.sum(people_mask)
                
                # Merge class 1 (people) into class 0 (pedestrian -> person)
                cls[people_mask] = 0
                
                # Shift classes > 1 down by 1
                gt1_mask = cls > 1
                shifted_count += np.sum(gt1_mask)
                cls[gt1_mask] -= 1
                
                # Store modified labels
                labels[i]['cls'] = cls
        
        # Now set correct class count and names for training
        if hasattr(self, 'data'):
            # Update names and class count
            self.data['names'] = self.merged_names
            self.data['nc'] = len(self.merged_names)
        
        # Log statistics
        person_count = sum(np.sum(label['cls'] == 0) for label in labels)
        LOGGER.info(f"\n{colorstr('VisDroneDataset:')} Remapped {people_count} 'people' instances to {self.merged_names[0]}")
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Total 'persona' instances after merge: {person_count}")
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Shifted {shifted_count} instances of other classes")
        
        return labels

class MergedClassDetectionTrainer(DetectionTrainer):
    
    """
    Custom trainer that uses VisDroneDataset for merged class training.
    """
    
    def build_dataset(self, img_path, mode="train", batch=None):
        """Build custom VisDroneDataset."""
        return VisDroneDataset(
            img_path=img_path,
            imgsz=self.args.imgsz,
            batch_size=batch or self.batch_size,
            augment=mode == "train",
            hyp=self.args,
            rect=self.args.rect if mode == "train" else True,
            cache=self.args.cache or None,
            single_cls=self.args.single_cls,
            stride=self.stride,
            pad=0.0 if mode == "train" else 0.5,
            prefix=colorstr(f"{mode}: "),
            task=self.args.task,
            classes=None,
            data=self.data,
            fraction=self.args.fraction if mode == "train" else 1.0,
        )
    
    def set_model_attributes(self):
        """Update model attributes for merged classes."""
        # First call parent method to set standard attributes
        super().set_model_attributes()
        
        # Then update model with the merged class names
        if hasattr(self.model, 'names'):
            # Use the merged names directly from the dataset class
            self.model.names = VisDroneDataset.merged_names
            self.model.nc = len(VisDroneDataset.merged_names)
            
            # Also update data dictionary
            if hasattr(self, 'data'):
                self.data['names'] = VisDroneDataset.merged_names
                self.data['nc'] = len(VisDroneDataset.merged_names)

class MergedClassDetectionValidator(DetectionValidator):
    """
    Custom validator that uses VisDroneDataset for validation/testing with merged classes.
    """
    
    def build_dataset(self, img_path, mode="val", batch=None):
        """Build custom VisDroneDataset for validation."""
        return VisDroneDataset(
            img_path=img_path,
            imgsz=self.args.imgsz,
            batch_size=batch or self.args.batch,
            augment=False,
            hyp=self.args,
            rect=True,
            cache=None,
            single_cls=self.args.single_cls,
            stride=self.stride,
            pad=0.5,
            prefix=colorstr(f"{mode}: "),
            task=self.args.task,
            classes=self.args.classes,
            data=self.data,
        )
    
    def set_model_attributes(self):
        """Update model attributes for merged classes if using a PyTorch model."""
        super().set_model_attributes()
        
        # Update model names if it's a PyTorch model (not for exported models)
        if hasattr(self.model, 'names') and hasattr(self.model, 'model'):
            self.model.names = VisDroneDataset.merged_names
            if hasattr(self.data, 'names'):
                self.data['names'] = VisDroneDataset.merged_names
                self.data['nc'] = len(VisDroneDataset.merged_names)

In [ ]:
# # Load top k samples from the top 15 configurations by fitness

# k = config["k_samples"]
# top_n = 15  # Number of top configurations to consider

# csv_path = "../data/processed/wandb_export_2025-03-05T10_24_46.923+01_00.csv"
# df = pd.read_csv(csv_path)

# # Calculate fitness score
# df['fitness'] = df['metrics/mAP50(B)'] * 0.1 + df['metrics/mAP50-95(B)'] * 0.9
# df = df.dropna(subset=['fitness'])

# # First, sort by fitness to get the top N configurations
# df_top = df.sort_values(by='fitness', ascending=False).head(top_n)

# # Then sample k configurations from these top performers
# df_sampled = df_top.sample(n=min(k, len(df_top)))

# columns_to_show = ['fitness', 'metrics/mAP50-95(B)', 'metrics/mAP50(B)', 
#                    'metrics/precision(B)', 'metrics/recall(B)', 'optimizer', 
#                    'lr0', 'lrf', 'momentum', 'weight_decay', 'box', 'cls', 'dfl']

# df_k_sampled = df_sampled[columns_to_show].reset_index(drop=True)
# print(f"Sampled {k} configs from top {top_n} by Fitness Score:")

# display(df_k_sampled)

In [ ]:
# Clear cache

def clear_cache():
    # Clear CUDA cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Clear Python garbage collector
    gc.collect()

In [ ]:
# Store results

def save_results(dir, name, results):
    os.makedirs(dir, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_path = f"{dir}/{name}_{timestamp}.json"

    with open(results_path, 'w') as f:
        json.dump(results, f, indent=4, default=str)
    
    print(f"{name} results saved to {results_path}")

In [ ]:
# # optimizer_step

# def optimizer_step(trainer):
#     """
#     Custom callback to implement cyclical learning rate schedule with parameter-specific learning rates.
#     This properly accounts for the Ultralytics optimizer parameter grouping where:
#     - Group 0: Weights with decay (not in BatchNorm layers)
#     - Group 1: BatchNorm weights (no decay)
#     - Group 2: Bias parameters (no decay)
#     """
#     # Only proceed if cyclic learning rate is enabled in config
#     if not trainer.data.get('cyclic_lr', {}).get('enabled', False):
#         return
    
#     # Get current epoch
#     epoch = trainer.epoch
    
#     # Get cyclic LR config parameters from the data dictionary
#     cyclic_config = trainer.data.get('cyclic_lr', {})
#     cycle_size = cyclic_config.get('cycle_size', 80)
#     lr_min_factor = cyclic_config.get('min_factor', 0.1)
#     cycle_decay = cyclic_config.get('decay', 0.85)
    
#     # Get initial learning rate from training args
#     initial_lr = trainer.args.lr0
    
#     # Calculate current cycle (1-based)
#     cycle = math.floor(epoch / cycle_size) + 1
    
#     # Calculate position within current cycle (0 to 1)
#     cycle_position = (epoch % cycle_size) / cycle_size
    
#     # Apply decay based on cycle number
#     cycle_decay_factor = cycle_decay ** (cycle - 1)
    
#     # Calculate LR factor using cosine annealing within each cycle
#     cosine_factor = 0.5 * (1 + math.cos(math.pi * cycle_position))
#     lr_range = 1.0 - lr_min_factor
#     lr_factor = (lr_min_factor + lr_range * cosine_factor) * cycle_decay_factor
    
#     # Base learning rate for this cycle
#     base_lr = initial_lr * lr_factor
    
#     # Set parameter-specific multipliers according to YOLO conventions
#     # Group 0: Weights with decay - standard learning rate
#     # Group 1: BatchNorm weights - can use higher learning rate
#     # Group 2: Bias parameters - typically higher learning rate
#     weight_decay_lr = base_lr * 1.0    # Standard rate for weights with decay
#     bn_lr = base_lr * 1.5              # Higher rate for BatchNorm (no decay)
#     bias_lr = base_lr * 2.0            # Even higher rate for bias terms
    
#     learning_rates = [weight_decay_lr, bn_lr, bias_lr]
#     group_names = ["Weights (with decay)", "BatchNorm weights", "Bias parameters"]
    
#     # Log parameter group information periodically
#     if epoch < 5 or epoch % 10 == 0:
#         LOGGER.info(f"\nEpoch {epoch}: Examining optimizer parameter groups:")
#         for i, pg in enumerate(trainer.optimizer.param_groups):
#             params_count = sum(p.numel() for p in pg['params'] if p.requires_grad)
#             current_lr = pg.get('lr', 'unknown')
#             LOGGER.info(f"  Group {i} ({group_names[i]}): {params_count} parameters, current lr: {current_lr}")
    
#     # Print the calculated learning rates
#     if epoch < 5 or epoch % 10 == 0:
#         LOGGER.info(f"  Calculated LRs: {[f'{lr:.6f}' for lr in learning_rates]}")
    
#     # Apply learning rates to each parameter group
#     for i, param_group in enumerate(trainer.optimizer.param_groups):
#         if i < len(learning_rates):
#             # Store previous learning rate for comparison
#             prev_lr = param_group.get('lr', 'unknown')
            
#             # Set new learning rate
#             param_group['lr'] = learning_rates[i]
            
#             # Verify change (in early epochs or periodically)
#             if epoch < 5 or epoch % 10 == 0:
#                 LOGGER.info(f"  Group {i} ({group_names[i]}): LR changed from {prev_lr} to {learning_rates[i]}")
    
#     # Log to wandb if available
#     if wandb.run is not None:
#         wandb.log({
#             "lr/weights_decay": learning_rates[0],
#             "lr/bn_weights": learning_rates[1],
#             "lr/bias": learning_rates[2],
#             "lr/cycle_factor": lr_factor,
#             "lr/cycle": cycle,
#             "lr/cycle_position": cycle_position,
#             "epoch": epoch
#         })
    
#     # Force an explicit print to ensure output is visible
#     sys.stdout.flush()

In [ ]:
def create_triangular_lr_callback(base_lr=0.001, max_lr=0.01, cycle_length=1000, group_scalers=None):
    """
    Creates a triangular learning rate callback.
    
    Args:
        base_lr (float): The minimum learning rate.
        max_lr (float): The maximum learning rate.
        cycle_length (int): Number of batches to complete one cycle.
        group_scalers (list, optional): Multipliers for each parameter group 
            (order: [weight decay, batchnorm (no decay), bias]). Default is [1.0, 1.0, 1.5].
    
    Returns:
        function: A callback function to be registered with model.add_callback().
    """
    if group_scalers is None:
        group_scalers = [1.0, 1.0, 1.5]
    
    def triangular_lr_callback(trainer, **kwargs):
        # Initialize the iteration counter if not already present.
        if not hasattr(trainer, 'lr_iteration'):
            trainer.lr_iteration = 0
        trainer.lr_iteration += 1

        # Calculate current position in the cycle.
        cycle_iter = trainer.lr_iteration % cycle_length
        half_cycle = cycle_length / 2

        # Determine scaling factor: increasing in first half, decreasing in second half.
        if cycle_iter <= half_cycle:
            factor = cycle_iter / half_cycle
        else:
            factor = (cycle_length - cycle_iter) / half_cycle

        # Compute the new base learning rate according to the triangular policy.
        new_lr = base_lr + (max_lr - base_lr) * factor

        # Update the learning rate for each optimizer parameter group.
        for i, param_group in enumerate(trainer.optimizer.param_groups):
            scaler = group_scalers[i] if i < len(group_scalers) else 1.0
            param_group['lr'] = new_lr * scaler

        # Optional: Uncomment the next line to print the learning rate for debugging.
        # print(f"Iteration {trainer.lr_iteration}: new base LR = {new_lr:.6f}")

    return triangular_lr_callback

In [ ]:
# Start training and validation

def start(model: YOLO, config):
    train_results = model.train(
        trainer=MergedClassDetectionTrainer,
        **config['train']
        )

    test_results = model.val(
        validator=MergedClassDetectionValidator,
        **config['val']
        )

    return train_results, test_results

In [ ]:
# Remove models

import glob
import os

def remove_models():
    pt_files = glob.glob("*.pt")
    print("Files to be removed:", pt_files)

    for file in pt_files:
        os.remove(file)

In [ ]:
# Wandb

def wandb_start(key, run_config, wandb_config):
    settings.update({"wandb": True})

    wandb.login(key=key, relogin=True)
    wandb.init(project=wandb_config["project"], group=wandb_config["group"])
    wandb.log(run_config)

In [ ]:
# Train

key = get_wandb_key()
settings.update({"wandb": True})
wandb.login(key=key, relogin=True)

for i in range(config["iterations"]):

    trial = config.copy()
    # trial.setdefault('cyclic_lr', {})['enabled'] = True
    
    trial["train"].update({
        "model": "yolo12l.pt",
        "pretrained": True,
        "imgsz": 1280,
        "optimizer": "AdamW",
        "lr0": 0.001,
        "lrf": 0.01,
        "warmup_epochs": 10,
        "patience": 5,
        "batch": 2,
        "workers": 2,
        "momentum": 0.937,
        "box": 3.5,
        "cls": 0.3,
        "dfl": 1,
        "cos_lr": False,
    })

    # trial.setdefault('cyclic_lr',{
    #     "base_lr": 0.001,
    #     "max_lr": 0.01,
    #     "cycle_size": 1000,   
    # })

    run = wandb.init(
        project=trial["wandb"]["project"], 
        group=trial["wandb"]["group"]
    )
    wandb.log(trial["train"])

    model = YOLO(trial["train"]["model"])

    # triangular_lr_cb = create_triangular_lr_callback(
    #         base_lr=trial["cyclic_lr"]["base_lr"], 
    #         max_lr=trial["cyclic_lr"]["max_lr"], 
    #         cycle_length=trial["cyclic_lr"]["cycle_size"]
    #     )
    # model.add_callback("on_train_batch_end", triangular_lr_cb)

    wandb_start(key, trial["train"], trial["wandb"])
    # wandb.log(trial["cyclic_lr"])

    train_results, test_results = start(model, trial)

    save_results("../data/processed", "train", train_results)
    save_results("../data/processed", "test", test_results)

    clear_cache()
    remove_models()

In [ ]:
# Resume

# resume_config = config.copy()
# del resume_config["val"]["name"]
# resume_config["train"].update({
#     "epochs": 300, 
#     "device": 0,
#     "warmup_epochs": 0,
#     "optimizer": "AdamW",
# })
# resume_config.update(df_train.iloc[0])
# resume_config